In [21]:
import os
from nltk.parse import stanford
import nltk
os.environ['STANFORD_PARSER'] = '/home/shanu/nltk/jars/stanford-parser.jar'
os.environ['STANFORD_MODELS'] = '/home/shanu/nltk/jars/stanford-parser-3.7.0-models.jar'

In [22]:
# Dependency Tree
from nltk.parse.stanford import StanfordDependencyParser
dep_parser=StanfordDependencyParser(model_path="/home/shanu/nltk/jars/englishPCFG.ser.gz")

In [23]:
def lca(tree, index1, index2):
    node = index1
    path1 = []
    path2 = []
    path1.append(index1)
    path2.append(index2)
    while(node != tree.root):
        node = tree.nodes[node['head']]
        path1.append(node)
    node = index2
    while(node != tree.root):
        node = tree.nodes[node['head']]
        path2.append(node)
    for l1, l2 in zip(path1[::-1],path2[::-1]):
        if(l1==l2):
            temp = l1
    return temp

In [24]:
def path_lca(tree, node, lca_node):
    path = []
    path.append(node)
    while(node != lca_node):
        node = tree.nodes[node['head']]
        path.append(node)
    return path

In [25]:
def seq(lca):
    l=[lca]
    for key in tree.nodes[lca]['deps']:
        for i in tree.nodes[lca]['deps'][key]:
            l.extend(seq(i))
    return l

In [28]:
import _pickle 
f = open('data/test_data', 'rb')
sentences, e1, e2 = _pickle.load(f)
f.close()

In [13]:
# sentences[7588] = 'The reaction mixture is kept in the dark at room temperature for 1.5 hours .'
# sentences[2608] = "This strawberry sauce has about a million uses , is freezer-friendly , and is so much better than that jar of Smuckers strawberry sauce that you 've had sitting in your fridge since that time you made banana splits 1.5 years ago ."

In [40]:
sentences[2590] = "The pendant with the bail measure 1.25'' ."

In [41]:
sentences[2664] = "The cabinet encloses a 6.5 inch cone woofer , 4 inch cone midrange , and a 0.86 inch balanced dome tweeter ."

In [42]:
length = len(sentences)

In [14]:
word_p = []
rel_p = []
pos_p = []
for i in range(length):
    word_p.append(0)
    rel_p.append(0)
    pos_p.append(0)

In [44]:
# for i in range(length):
i = 2590
try:
    parse_tree = dep_parser.raw_parse(sentences[i])
    for trees in parse_tree:
        tree = trees
    node1 = tree.nodes[e1[i]+1]
    node2 = tree.nodes[e2[i]+1]
    if node1['address']!=None and node2['address']!=None:
        lca_node = lca(tree, node1, node2)
        path = seq(lca_node['address'])
        print(i, "success", path)

        word_p[i] = [tree.nodes[p]["word"] for p in path]
        rel_p[i] = [tree.nodes[p]["rel"] for p in path]
        pos_p[i] = [tree.nodes[p]["tag"] for p in path]
    else:

        print(i, node1["address"], node2["address"])
except AssertionError:
    print(i, "error")
    

2590 success [2, 1, 6, 4, 5, 3]


In [45]:
file = open('data/test_lca_paths', 'wb')
_pickle.dump([word_p, rel_p, pos_p], file)

In [39]:
file = open('data/test_lca_paths', 'rb')
word_p, rel_p, pos_p  = _pickle.load(file)